In [1]:
import pandas as pd;
import numpy as np;
from scipy import spatial;
import collections;
from sklearn.metrics import jaccard_score
from sklearn.metrics import mean_squared_error
import math

cols=["user_id","movie_id","ratings","timestamp"]
df=pd.read_csv("u1.base",sep="\t",names=cols,encoding="latin-1")
test_df=pd.read_csv("u1.test",sep="\t",names=cols,encoding="latin-1")
user_user_test_df=pd.read_csv("u1.test",sep="\t",names=cols,encoding="latin-1")
movie_movie_test_df=pd.read_csv("u1.test",sep="\t",names=cols,encoding="latin-1")
user_movie_mat=np.zeros((df["user_id"].max()+1,df["movie_id"].max()+1));
movie_user_mat=np.zeros((df["movie_id"].max()+1,df["user_id"].max()+1));
user_movie_mat[df["user_id"],df["movie_id"]]=df['ratings']
movie_user_mat[df["movie_id"],df["user_id"]]=df['ratings']
movie_user_boolean_mat = movie_user_mat>0
user_movie_boolean_mat = user_movie_mat>0
test_dataframe=pd.read_csv("u1.test",sep="\t",names=cols,encoding="latin-1")
overall_mean_rating = test_dataframe['ratings'].mean();

avg_rating_user_group = test_dataframe.groupby("user_id");
avg_rating_movie_group = test_dataframe.groupby("movie_id");
avg_rating_user = avg_rating_user_group['ratings'].mean().reset_index()
avg_rating_movie = avg_rating_movie_group['ratings'].mean().reset_index()

In [2]:
from sortedcontainers import SortedList, SortedSet, SortedDict
import copy

In [4]:
avg_rating_user = test_df.groupby("user_id");
avg_rating_movie = test_df.groupby("movie_id");
overall_mean_rating = test_df['ratings'].mean();
avg_rating_user = avg_rating_user['ratings'].mean().reset_index()
avg_rating_movie = avg_rating_movie['ratings'].mean().reset_index()

In [5]:
def getTopKRatings(k,sim_array,user_movies,ratings_df,index):
    elem=np.copy(sim_array)
    elem.sort()
    num=0;
    den=0;
    for i in range(1,k+1):
        sim = elem[-(i+1)];
        den = den+sim;
        similarElemem = np.where(sim_array==sim);
        candidateMovies = np.intersect1d(user_movies,similarElemem);
        for mov in candidateMovies:
            mov_rating = ratings_df.loc[ratings_df[index]==mov]["ratings"].to_numpy()
            if mov_rating > 0 and sim > 0:
                num = num + (float(mov_rating) * sim)
    if num==0 or den==0:
        return 0;
    return num/den;

def flatten_tuple(data):
    if isinstance(data, tuple):
        if len(data) == 0:
            return ()
        else:
            return flatten_tuple(data[0]) + flatten_tuple(data[1:])
    else:
        return (data,)

In [7]:
def getBaisedTopKRatings(k,sim_array,user_movies,ratings_df,index,index_avg_ratings,avg_rating_of_target):
    elem=np.copy(sim_array)
    elem.sort()
    num=0;
    den=0;
    for i in range(1,k+1):
        sim = elem[-(i+1)];
        similarElemem = np.where(sim_array==sim);
        candidates = np.intersect1d(user_movies,similarElemem);
        #candidates will be set of movies watched by the user in case of item-item
        #candidates will be set of user who watched the movie in case of user-user
        for candidate in candidates:
            
            rating = ratings_df.loc[ratings_df[index]==candidate]["ratings"].to_numpy()
            avgRatingOfCandidate = avg_rating_of_target.loc[avg_rating_of_target[index]==candidate]["ratings"].to_numpy()[0]
            
            bias = GetBias(overall_mean_rating,index_avg_ratings,avgRatingOfCandidate)
            
            if rating > 0 and sim > 0:
                num = num + ((float(rating) - bias) * sim)
                den = den+sim;
    if ((num==0) or (den==0)):
        return 0;
    #print(num,den);
    return num/den;
        

In [9]:
def getAvgPrecisionAt10(actual_df,predicted_key,group_by_key,piviot_key,k=2.5):
    actual_df_grouping=actual_df.loc[actual_df['ratings']>k].groupby([group_by_key])
    #Considering only Actual Value , we donot consider the predicted value
    predicted_df_grouping=actual_df.groupby([group_by_key])
    
    #In case we consider predicted value also 
    #predicted_df_grouping=actual_df.loc[actual_df[predicted_key]>k].groupby([group_by_key])
    
    total=0;
    cnt=0;
    for name,group in actual_df_grouping:
        actual = actual_df_grouping.get_group(name).head(10)[piviot_key].to_numpy()
        if name in predicted_df_grouping.groups.keys():
            #predicted = predicted_df_grouping.get_group(name).head(10)[piviot_key].to_numpy()
            predicted = predicted_df_grouping.get_group(name)[piviot_key].to_numpy();
            predicted.sort()
            predicted = predicted[-10:];
            union = len(np.union1d(actual,predicted));
            intersection = len(np.intersect1d(actual,predicted));
            if union>0 and intersection>0:          
                total+=(intersection/union);
        cnt+=1;    
        
    if total >0:
        return total/cnt;
    return 0;



In [10]:
#here index_avg_ratings can be user_avg_rating in case of item-item or movie_avg_rating in case of user-user
# avg_rating_of_pivot will be avg rating of movie i , in case of item-item and avg rating of user x in case of user-user
def GetBias(overall_mean_rating,index_avg_ratings,avg_rating_of_pivot):
    return index_avg_ratings+avg_rating_of_pivot-overall_mean_rating;
    

# Train Models

In [11]:


#Jaccard Similarity

user_user_jaccard_similarity_mat = np.zeros((df["user_id"].max()+1,df["user_id"].max()+1))
for i in range(1,len(user_movie_mat)):
    for j in range(1,len(user_movie_mat)):
        user_user_jaccard_similarity_mat[i][j] = 1 - spatial.distance.jaccard(user_movie_boolean_mat[i], user_movie_boolean_mat[j])

movie_movie_jaccard_similarity_mat = np.zeros((df["movie_id"].max()+1,df["movie_id"].max()+1))
for i in range(1,len(movie_user_mat)):
    for j in range(1,len(movie_user_mat)):
        movie_movie_jaccard_similarity_mat[i][j] = 1 - spatial.distance.jaccard(movie_user_boolean_mat[i], movie_user_boolean_mat[j])
        


#Cosine Similarity
user_user_consine_similarity_mat = np.zeros((df["user_id"].max()+1,df["user_id"].max()+1))
for i in range(1,len(user_movie_mat)):
    for j in range(1,len(user_movie_mat)):
        user_user_consine_similarity_mat[i][j] = 1 - spatial.distance.cosine(user_movie_mat[i], user_movie_mat[j])


movie_movie_consine_similarity_mat = np.zeros((df["movie_id"].max()+1,df["movie_id"].max()+1))
for i in range(1,len(movie_user_mat)):
    for j in range(1,len(movie_user_mat)):
        movie_movie_consine_similarity_mat[i][j] = 1 - spatial.distance.cosine(movie_user_mat[i], movie_user_mat[j])
        
#Pearson Similarity

user_user_pearson_similarity_mat = np.zeros((df["user_id"].max()+1,df["user_id"].max()+1))
for i in range(1,len(user_movie_mat)):
    for j in range(1,len(user_movie_mat)):
        user_user_pearson_similarity_mat[i][j] = 1 - spatial.distance.correlation(user_movie_mat[i], user_movie_mat[j])

movie_movie_pearson_similarity_mat = np.zeros((df["movie_id"].max()+1,df["movie_id"].max()+1))
for i in range(1,len(movie_user_mat)):
    for j in range(1,len(movie_user_mat)):
        movie_movie_pearson_similarity_mat[i][j] = 1 - spatial.distance.correlation(movie_user_mat[i], movie_user_mat[j])
  




C:\Users\rcranjan.ORADEV\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\spatial\distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


# Find Rating in Test Data

In [13]:
#Jaccard find rating 
tempDict={}
for i in range(1,test_df["movie_id"].max()+1):
    tempDict[i]={}
    tempDict[i]['Users']=test_df.loc[test_df['movie_id']==i]['user_id'].unique()

user_user_jaccard_similarity_mat=np.nan_to_num(user_user_jaccard_similarity_mat)

    
for i in range(1,test_df["movie_id"].max()+1):
    movie_df = movie_movie_test_df.loc[movie_movie_test_df["movie_id"]==i];
    for j in tempDict[i]['Users']:
        user_user_test_df.loc[((user_user_test_df["movie_id"]==i) & (user_user_test_df["user_id"]==j)),"jaccard_calculatedTop2"] = getTopKRatings(2,user_user_jaccard_similarity_mat[j],tempDict[i]['Users'],movie_df,"user_id");
        user_user_test_df.loc[((user_user_test_df["movie_id"]==i) & (user_user_test_df["user_id"]==j)),"jaccard_calculatedTop5"] = getTopKRatings(5,user_user_jaccard_similarity_mat[j],tempDict[i]['Users'],movie_df,"user_id");
        user_user_test_df.loc[((user_user_test_df["movie_id"]==i) & (user_user_test_df["user_id"]==j)),"jaccard_calculatedTop10"] = getTopKRatings(10,user_user_jaccard_similarity_mat[j],tempDict[i]['Users'],movie_df,"user_id");


tempDict={}
for i in range(1,test_df["user_id"].max()+1):
    tempDict[i]={}
    tempDict[i]['Movies']=test_df.loc[test_df['user_id']==i]['movie_id'].unique()

movie_movie_jaccard_similarity_mat=np.nan_to_num(movie_movie_jaccard_similarity_mat)

    
for i in range(1,test_df["user_id"].max()+1):
    user_df = test_df.loc[test_df["user_id"]==i];
    ns=0;
    ds=0;
    for j in tempDict[i]['Movies']:
        test_df.loc[((test_df["user_id"]==i) & (test_df["movie_id"]==j)),"jaccard_calculatedTop2"] = getTopKRatings(2,movie_movie_jaccard_similarity_mat[j],tempDict[i]['Movies'],user_df,"movie_id");
        test_df.loc[((test_df["user_id"]==i) & (test_df["movie_id"]==j)),"jaccard_calculatedTop5"] = getTopKRatings(5,movie_movie_jaccard_similarity_mat[j],tempDict[i]['Movies'],user_df,"movie_id");
        test_df.loc[((test_df["user_id"]==i) & (test_df["movie_id"]==j)),"jaccard_calculatedTop10"] = getTopKRatings(10,movie_movie_jaccard_similarity_mat[j],tempDict[i]['Movies'],user_df,"movie_id");
        
#Pearson find rating 
movie_movie_pearson_similarity_mat=np.nan_to_num(movie_movie_pearson_similarity_mat)

tempDict={}
for i in range(1,test_df["user_id"].max()+1):
    tempDict[i]={}
    tempDict[i]['Movies']=test_df.loc[test_df['user_id']==i]['movie_id'].unique()

movie_movie_pearson_similarity_mat==np.nan_to_num(movie_movie_pearson_similarity_mat)
        
for i in range(1,test_df["user_id"].max()+1):
    user_df = test_df.loc[test_df["user_id"]==i];
    ns=0;
    ds=0;
    for j in tempDict[i]['Movies']:
        test_df.loc[((test_df["user_id"]==i) & (test_df["movie_id"]==j)),"pearson_calculatedTop2"] = getTopKRatings(2,movie_movie_pearson_similarity_mat[j],tempDict[i]['Movies'],user_df,"movie_id");
        test_df.loc[((test_df["user_id"]==i) & (test_df["movie_id"]==j)),"pearson_calculatedTop5"] = getTopKRatings(5,movie_movie_pearson_similarity_mat[j],tempDict[i]['Movies'],user_df,"movie_id");
        test_df.loc[((test_df["user_id"]==i) & (test_df["movie_id"]==j)),"pearson_calculatedTop10"] = getTopKRatings(10,movie_movie_pearson_similarity_mat[j],tempDict[i]['Movies'],user_df,"movie_id");

tempDict={}
for i in range(1,test_df["movie_id"].max()+1):
    tempDict[i]={}
    tempDict[i]['Users']=test_df.loc[test_df['movie_id']==i]['user_id'].unique()

user_user_pearson_similarity_mat=np.nan_to_num(user_user_pearson_similarity_mat)

    
for i in range(1,test_df["movie_id"].max()+1):
    movie_df = movie_movie_test_df.loc[movie_movie_test_df["movie_id"]==i];
    for j in tempDict[i]['Users']:
       
        user_user_test_df.loc[((user_user_test_df["movie_id"]==i) & (user_user_test_df["user_id"]==j)),"pearson_calculatedTop2"] = getTopKRatings(2,user_user_pearson_similarity_mat[j],tempDict[i]['Users'],movie_df,"user_id");
        user_user_test_df.loc[((user_user_test_df["movie_id"]==i) & (user_user_test_df["user_id"]==j)),"pearson_calculatedTop5"] = getTopKRatings(5,user_user_pearson_similarity_mat[j],tempDict[i]['Users'],movie_df,"user_id");
        user_user_test_df.loc[((user_user_test_df["movie_id"]==i) & (user_user_test_df["user_id"]==j)),"pearson_calculatedTop10"] = getTopKRatings(10,user_user_pearson_similarity_mat[j],tempDict[i]['Users'],movie_df,"user_id");
                
        
#Cosine Find Rating
tempDict={}
for i in range(1,test_df["movie_id"].max()+1):
    tempDict[i]={}
    tempDict[i]['Users']=test_df.loc[test_df['movie_id']==i]['user_id'].unique()

user_user_consine_similarity_mat=np.nan_to_num(user_user_consine_similarity_mat)

    
for i in range(1,test_df["movie_id"].max()+1):
    movie_df = movie_movie_test_df.loc[movie_movie_test_df["movie_id"]==i];
    for j in tempDict[i]['Users']:
        #val = getTopKRatings(2,user_user_consine_similarity_mat[j],i,j,movie_df,"user_id");
        #print(val)
        user_user_test_df.loc[((user_user_test_df["movie_id"]==i) & (user_user_test_df["user_id"]==j)),"cosine_calculatedTop2"] = getTopKRatings(2,user_user_consine_similarity_mat[j],tempDict[i]['Users'],movie_df,"user_id");
        user_user_test_df.loc[((user_user_test_df["movie_id"]==i) & (user_user_test_df["user_id"]==j)),"cosine_calculatedTop5"] = getTopKRatings(5,user_user_consine_similarity_mat[j],tempDict[i]['Users'],movie_df,"user_id");
        user_user_test_df.loc[((user_user_test_df["movie_id"]==i) & (user_user_test_df["user_id"]==j)),"cosine_calculatedTop10"] = getTopKRatings(10,user_user_consine_similarity_mat[j],tempDict[i]['Users'],movie_df,"user_id");
        
tempDict={}
for i in range(1,test_df["user_id"].max()+1):
    tempDict[i]={}
    tempDict[i]['Movies']=test_df.loc[test_df['user_id']==i]['movie_id'].unique()

movie_movie_consine_similarity_mat=np.nan_to_num(movie_movie_consine_similarity_mat)

    
for i in range(1,test_df["user_id"].max()+1):
    user_df = test_df.loc[test_df["user_id"]==i];
    ns=0;
    ds=0;
    for j in tempDict[i]['Movies']:
        test_df.loc[((test_df["user_id"]==i) & (test_df["movie_id"]==j)),"cosine_calculatedTop2"] = getTopKRatings(2,movie_movie_consine_similarity_mat[j],tempDict[i]['Movies'],user_df,"movie_id");
        test_df.loc[((test_df["user_id"]==i) & (test_df["movie_id"]==j)),"cosine_calculatedTop5"] = getTopKRatings(5,movie_movie_consine_similarity_mat[j],tempDict[i]['Movies'],user_df,"movie_id");
        test_df.loc[((test_df["user_id"]==i) & (test_df["movie_id"]==j)),"cosine_calculatedTop10"] = getTopKRatings(10,movie_movie_consine_similarity_mat[j],tempDict[i]['Movies'],user_df,"movie_id");
 


In [15]:
threashold=4
print("********************** QUESTION 1 ********************************************************")
print("********************** Threashold ",threashold,"********* for Precision@10 **************");
print("************ user user Cosine ***********************")
print("Cosine User User Top2_RMSE_Score  : ",math.sqrt(mean_squared_error(user_user_test_df['cosine_calculatedTop2'],user_user_test_df['ratings'])))
print("Cosine User User Top5_RMSE_Score  : ",math.sqrt(mean_squared_error(user_user_test_df['cosine_calculatedTop5'],user_user_test_df['ratings'])))
print("Cosine User User Top10_RMSE_Score : ",math.sqrt(mean_squared_error(user_user_test_df['cosine_calculatedTop10'],user_user_test_df['ratings'])))
print("Cosine User User Top2_Precision@10 : ",getAvgPrecisionAt10(user_user_test_df,'cosine_calculatedTop2','user_id','movie_id',threashold));
print("Cosine User User Top5_Precision@10 : ",getAvgPrecisionAt10(user_user_test_df,'cosine_calculatedTop5','user_id','movie_id',threashold));
print("Cosine User User Top10_Precision@10 : ",getAvgPrecisionAt10(user_user_test_df,'cosine_calculatedTop10','user_id','movie_id',threashold));
print("\n")
print("************ Movie Movie Cosine ***********************")
print("Cosine Movie Movie Top2_RMSE_Score  : ",math.sqrt(mean_squared_error(test_df['cosine_calculatedTop2'],test_df['ratings'])))
print("Cosine Movie Movie Top5_RMSE_Score  : ",math.sqrt(mean_squared_error(test_df['cosine_calculatedTop5'],test_df['ratings'])))
print("Cosine Movie Movie Top10_RMSE_Score : ",math.sqrt(mean_squared_error(test_df['cosine_calculatedTop10'],test_df['ratings'])))
print("Cosine Movie Movie Top2_Precision@10 : ",getAvgPrecisionAt10(test_df,'cosine_calculatedTop2','movie_id','user_id',threashold));
print("Cosine Movie Movie Top5_Precision@10 : ",getAvgPrecisionAt10(test_df,'cosine_calculatedTop5','movie_id','user_id',threashold));
print("Cosine Movie Movie Top10_Precision@10 : ",getAvgPrecisionAt10(test_df,'cosine_calculatedTop10','movie_id','user_id',threashold));

print("\n")
print("********************************************************************")
print("************ user user Pearson ***********************")
print("Pearson Top2_RMSE_Score  : ",math.sqrt(mean_squared_error(user_user_test_df['pearson_calculatedTop2'],user_user_test_df['ratings'])))
print("Pearson Top5_RMSE_Score  : ",math.sqrt(mean_squared_error(user_user_test_df['pearson_calculatedTop5'],user_user_test_df['ratings'])))
print("Pearson Top10_RMSE_Score : ",math.sqrt(mean_squared_error(user_user_test_df['pearson_calculatedTop10'],user_user_test_df['ratings'])))
print("Pearson User User Top2_Precision@10 : ",getAvgPrecisionAt10(user_user_test_df,'pearson_calculatedTop2','user_id','movie_id',threashold));
print("Pearson User User Top5_Precision@10 : ",getAvgPrecisionAt10(user_user_test_df,'pearson_calculatedTop5','user_id','movie_id',threashold));
print("Pearson User User Top10_Precision@10 : ",getAvgPrecisionAt10(user_user_test_df,'pearson_calculatedTop10','user_id','movie_id',threashold));

print("\n")
print("************ Movie Movie Pearson ***********************")
print("Pearson Top2_RMSE_Score  : ",math.sqrt(mean_squared_error(test_df['pearson_calculatedTop2'],test_df['ratings'])))
print("Pearson Top5_RMSE_Score  : ",math.sqrt(mean_squared_error(test_df['pearson_calculatedTop5'],test_df['ratings'])))
print("Pearson Top10_RMSE_Score : ",math.sqrt(mean_squared_error(test_df['pearson_calculatedTop10'],test_df['ratings'])))
print("Pearson Movie Movie Top2_Precision@10 : ",getAvgPrecisionAt10(test_df,'pearson_calculatedTop2','movie_id','user_id',threashold));
print("Pearson Movie Movie Top5_Precision@10 : ",getAvgPrecisionAt10(test_df,'pearson_calculatedTop5','movie_id','user_id',threashold));
print("Pearson Movie Movie Top10_Precision@10 : ",getAvgPrecisionAt10(test_df,'pearson_calculatedTop10','movie_id','user_id',threashold));

print("\n")
print("********************************************************************")
print("************ user user Jaccard ***********************")
print("Jaccard Top2_RMSE_Score  : ",((user_user_test_df['jaccard_calculatedTop2']-user_user_test_df['ratings'])**2).mean() ** .5)
print("Jaccard Top5_RMSE_Score  : ",((user_user_test_df['jaccard_calculatedTop5']-user_user_test_df['ratings'])**2).mean() ** .5)
print("Jaccard Top10_RMSE_Score : ",((user_user_test_df['jaccard_calculatedTop10']-user_user_test_df['ratings'])**2).mean() ** .5)
print("Jaccard user user Top2_Precision@10 : ",getAvgPrecisionAt10(user_user_test_df,'pearson_calculatedTop2','user_id','movie_id',threashold));
print("Jaccard user user Top5_Precision@10 : ",getAvgPrecisionAt10(user_user_test_df,'pearson_calculatedTop5','user_id','movie_id',threashold));
print("Jaccard user user Top10_Precision@10 : ",getAvgPrecisionAt10(user_user_test_df,'pearson_calculatedTop10','user_id','movie_id',threashold));

print("\n")
print("************ Movie Movie Jaccard ***********************")
print("Jaccard Top2_RMSE_Score  : ",math.sqrt(mean_squared_error(test_df['jaccard_calculatedTop2'],test_df['ratings'])))
print("Jaccard Top5_RMSE_Score  : ",math.sqrt(mean_squared_error(test_df['jaccard_calculatedTop5'],test_df['ratings'])))
print("Jaccard Top10_RMSE_Score : ",math.sqrt(mean_squared_error(test_df['jaccard_calculatedTop2'],test_df['ratings'])))
print("Jaccard Movie Movie Top2_Precision@10 : ",getAvgPrecisionAt10(test_df,'pearson_calculatedTop2','movie_id','user_id',threashold));
print("Jaccard Movie Movie Top5_Precision@10 : ",getAvgPrecisionAt10(test_df,'pearson_calculatedTop5','movie_id','user_id',threashold));
print("Jaccard Movie Movie Top10_Precision@10 : ",getAvgPrecisionAt10(test_df,'pearson_calculatedTop10','movie_id','user_id',threashold));

print("********************************************************************")


********************** QUESTION 1 ********************************************************
********************** Threashold  4 ********* for Precision@10 **************
************ user user Cosine ***********************
Cosine User User Top2_RMSE_Score  :  3.6794073109841103
Cosine User User Top5_RMSE_Score  :  3.667787915665507
Cosine User User Top10_RMSE_Score :  3.6590221814523103
Cosine User User Top2_Precision@10 :  0.13232733426967275
Cosine User User Top5_Precision@10 :  0.13232733426967275
Cosine User User Top10_Precision@10 :  0.13232733426967275


************ Movie Movie Cosine ***********************
Cosine Movie Movie Top2_RMSE_Score  :  3.040347475878824
Cosine Movie Movie Top5_RMSE_Score  :  2.936625640045252
Cosine Movie Movie Top10_RMSE_Score :  2.9131421806583737
Cosine Movie Movie Top2_Precision@10 :  0.20998117332502944
Cosine Movie Movie Top5_Precision@10 :  0.20998117332502944
Cosine Movie Movie Top10_Precision@10 :  0.20998117332502944


*********************

In [16]:
def getNAverageRating(N,ratings):
    rating_sum=0;
    sorted_ratings = np.sort(ratings)
    for i in range(1,N+1):
        if len(sorted_ratings) >= (i+1):
            rating_sum = rating_sum + sorted_ratings[-i];
    if rating_sum>0 :
        return rating_sum/N;
    return 0;

In [17]:
threashold=4


# find Average rating 
tempDict={}
for i in range(1,df["user_id"].max()+1):
    tempDict[i]={}
    tempDict[i]['Movies']=test_df.loc[test_df['user_id']==i]['movie_id'].unique()

movie_movie_consine_similarity_mat=np.nan_to_num(movie_movie_consine_similarity_mat)

    
for i in range(1,df["user_id"].max()+1):
    user_df = test_df.loc[test_df["user_id"]==i];
    for j in tempDict[i]['Movies']:
        test_df.loc[((test_df["user_id"]==i) & (test_df["movie_id"]==j)),"avg_calculatedTop2"] = getNAverageRating(2,user_df['ratings'].to_numpy());
        test_df.loc[((test_df["user_id"]==i) & (test_df["movie_id"]==j)),"avg_calculatedTop5"] = getNAverageRating(5,user_df['ratings'].to_numpy());
        test_df.loc[((test_df["user_id"]==i) & (test_df["movie_id"]==j)),"avg_calculatedTop10"] = getNAverageRating(10,user_df['ratings'].to_numpy());
 
# find Average rating 

tempDict={}
for i in range(1,df["movie_id"].max()+1):
    tempDict[i]={}
    tempDict[i]['Users']=test_df.loc[test_df['movie_id']==i]['user_id'].unique()

user_user_pearson_similarity_mat=np.nan_to_num(user_user_pearson_similarity_mat)

    
for i in range(1,df["movie_id"].max()+1):
    movie_df = movie_movie_test_df.loc[movie_movie_test_df["movie_id"]==i];
    for j in tempDict[i]['Users']:
       
        user_user_test_df.loc[((user_user_test_df["movie_id"]==i) & (user_user_test_df["user_id"]==j)),"avg_calculatedTop2"] = getNAverageRating(2,movie_df['ratings'].to_numpy());
        user_user_test_df.loc[((user_user_test_df["movie_id"]==i) & (user_user_test_df["user_id"]==j)),"avg_calculatedTop5"] = getNAverageRating(5,movie_df['ratings'].to_numpy());
        user_user_test_df.loc[((user_user_test_df["movie_id"]==i) & (user_user_test_df["user_id"]==j)),"avg_calculatedTop10"] = getNAverageRating(10,movie_df['ratings'].to_numpy());


# find Pearson Bias rating 
tempDict={}
for i in range(1,test_df["user_id"].max()+1):
    tempDict[i]={}
    tempDict[i]['Movies']=test_df.loc[test_df['user_id']==i]['movie_id'].unique()

movie_movie_pearson_similarity_mat=np.nan_to_num(movie_movie_pearson_similarity_mat)


for i in range(1,test_df["user_id"].max()+1):
    movie_df = test_df.loc[test_df["user_id"]==i];
    if movie_df.empty:
        continue;
    avgUserRating = avg_rating_user.loc[avg_rating_user.user_id==i]['ratings'].to_numpy()[0];
    
    for j in tempDict[i]['Movies']:
        avgMovieRating = avg_rating_movie.loc[avg_rating_movie.movie_id==j]['ratings'].to_numpy()[0]
        bias =  GetBias(overall_mean_rating,avgUserRating,avgMovieRating)
        test_df.loc[((test_df["user_id"]==i) & (test_df["movie_id"]==j)),"biased_pearson_Top2"] = bias+getBaisedTopKRatings(2,movie_movie_pearson_similarity_mat[j],tempDict[i]['Movies'],movie_df,"movie_id",avgUserRating,avg_rating_movie);
        test_df.loc[((test_df["user_id"]==i) & (test_df["movie_id"]==j)),"biased_pearson_Top5"] = bias+getBaisedTopKRatings(5,movie_movie_pearson_similarity_mat[j],tempDict[i]['Movies'],movie_df,"movie_id",avgUserRating,avg_rating_movie);
        test_df.loc[((test_df["user_id"]==i) & (test_df["movie_id"]==j)),"biased_pearson_Top10"] = bias+getBaisedTopKRatings(10,movie_movie_pearson_similarity_mat[j],tempDict[i]['Movies'],movie_df,"movie_id",avgUserRating,avg_rating_movie);

tempDict={}
for i in range(1,df["movie_id"].max()+1):
    tempDict[i]={}
    tempDict[i]['Users']=user_user_test_df.loc[user_user_test_df['movie_id']==i]['user_id'].unique()

user_user_pearson_similarity_mat=np.nan_to_num(user_user_pearson_similarity_mat)
    
for i in range(1,user_user_test_df["movie_id"].max()+1):
    user_df = test_df.loc[test_df["movie_id"]==i];
    if user_df.empty:
        continue;
    avgMovieRating = avg_rating_movie.loc[avg_rating_movie.movie_id==i]['ratings'].to_numpy()[0]
    for j in tempDict[i]['Users']:
        avgUserRating = avg_rating_user.loc[avg_rating_user.user_id==j]['ratings'].to_numpy()[0];
        bias =  GetBias(overall_mean_rating,avgUserRating,avgMovieRating)
        user_user_test_df.loc[((user_user_test_df["movie_id"]==i) & (user_user_test_df["user_id"]==j)),"biased_pearson_Top2"] = bias+getBaisedTopKRatings(2,user_user_pearson_similarity_mat[j],tempDict[i]['Users'],user_df,"user_id",avgUserRating,avg_rating_user);
        user_user_test_df.loc[((user_user_test_df["movie_id"]==i) & (user_user_test_df["user_id"]==j)),"biased_pearson_Top5"] = bias+getBaisedTopKRatings(5,user_user_pearson_similarity_mat[j],tempDict[i]['Users'],user_df,"user_id",avgUserRating,avg_rating_user);
        user_user_test_df.loc[((user_user_test_df["movie_id"]==i) & (user_user_test_df["user_id"]==j)),"biased_pearson_Top10"] = bias+getBaisedTopKRatings(10,user_user_pearson_similarity_mat[j],tempDict[i]['Users'],user_df,"user_id",avgUserRating,avg_rating_user);


In [19]:
threashold=4
print("********************** QUESTION 2 ********************************************************")
print("********************** Threashold ",threashold,"********* for Precision@10 **************");
print("************ user user Avg ***********************")
print("Avg User User Top2_RMSE_Score  : ",math.sqrt(mean_squared_error(user_user_test_df['avg_calculatedTop2'],user_user_test_df['ratings'])))
print("Avg User User Top5_RMSE_Score  : ",math.sqrt(mean_squared_error(user_user_test_df['avg_calculatedTop5'],user_user_test_df['ratings'])))
print("Avg User User Top10_RMSE_Score : ",math.sqrt(mean_squared_error(user_user_test_df['avg_calculatedTop10'],user_user_test_df['ratings'])))
print("Avg User User Top2_Precision@10 : ",getAvgPrecisionAt10(user_user_test_df,'avg_calculatedTop2','user_id','movie_id',threashold));
print("Avg User User Top5_Precision@10 : ",getAvgPrecisionAt10(user_user_test_df,'avg_calculatedTop5','user_id','movie_id',threashold));
print("Avg User User Top10_Precision@10 : ",getAvgPrecisionAt10(user_user_test_df,'avg_calculatedTop10','user_id','movie_id',threashold));
print("\n")
print("************ Movie Movie Avg ***********************")
print("Avg Movie Movie Top2_RMSE_Score  : ",math.sqrt(mean_squared_error(test_df['avg_calculatedTop2'],test_df['ratings'])))
print("Avg Movie Movie Top5_RMSE_Score  : ",math.sqrt(mean_squared_error(test_df['avg_calculatedTop5'],test_df['ratings'])))
print("Avg Movie Movie User Top10_RMSE_Score : ",math.sqrt(mean_squared_error(test_df['avg_calculatedTop10'],test_df['ratings'])))
print("Avg Movie Movie Top2_Precision@10 : ",getAvgPrecisionAt10(test_df,'avg_calculatedTop2','movie_id','user_id',threashold));
print("Avg Movie Movie Top5_Precision@10 : ",getAvgPrecisionAt10(test_df,'avg_calculatedTop5','movie_id','user_id',threashold));
print("Avg Movie Movie Top10_Precision@10 : ",getAvgPrecisionAt10(test_df,'avg_calculatedTop10','movie_id','user_id',threashold));
print("\n")
print("********************************************************************")
print("************ user user Pearson ***********************")
print("Pearson User User Top2_RMSE_Score  : ",math.sqrt(mean_squared_error(user_user_test_df['pearson_calculatedTop2'],user_user_test_df['ratings'])))
print("Pearson User User Top5_RMSE_Score  : ",math.sqrt(mean_squared_error(user_user_test_df['pearson_calculatedTop5'],user_user_test_df['ratings'])))
print("Pearson User User Top10_RMSE_Score : ",math.sqrt(mean_squared_error(user_user_test_df['pearson_calculatedTop10'],user_user_test_df['ratings'])))
print("Pearson User User Top2_Precision@10 : ",getAvgPrecisionAt10(user_user_test_df,'pearson_calculatedTop2','user_id','movie_id',threashold));
print("Pearson User User Top5_Precision@10 : ",getAvgPrecisionAt10(user_user_test_df,'pearson_calculatedTop5','user_id','movie_id',threashold));
print("Pearson User User Top10_Precision@10 : ",getAvgPrecisionAt10(user_user_test_df,'pearson_calculatedTop10','user_id','movie_id',threashold));

print("\n")
print("************ Movie Movie Pearson ***********************")
print("Pearson Movie Movie Top2_RMSE_Score  : ",math.sqrt(mean_squared_error(test_df['pearson_calculatedTop2'],test_df['ratings'])))
print("Pearson Movie Movie Top5_RMSE_Score  : ",math.sqrt(mean_squared_error(test_df['pearson_calculatedTop5'],test_df['ratings'])))
print("Pearson Movie Movie Top10_RMSE_Score : ",math.sqrt(mean_squared_error(test_df['pearson_calculatedTop10'],test_df['ratings'])))
print("Pearson Movie Movie Top2_Precision@10 : ",getAvgPrecisionAt10(test_df,'pearson_calculatedTop2','movie_id','user_id',threashold));
print("Pearson Movie Movie Top5_Precision@10 : ",getAvgPrecisionAt10(test_df,'pearson_calculatedTop5','movie_id','user_id',threashold));
print("Pearson Movie Movie Top10_Precision@10 : ",getAvgPrecisionAt10(test_df,'pearson_calculatedTop10','movie_id','user_id',threashold));
print("\n")
print("********************************************************************")
print("************ user user Bias ***********************")
print("Pearson User User Bias Top2_RMSE_Score  : ",math.sqrt(mean_squared_error(user_user_test_df['biased_pearson_Top2'],user_user_test_df['ratings'])))
print("Pearson User User Bias Top5_RMSE_Score  : ",math.sqrt(mean_squared_error(user_user_test_df['biased_pearson_Top5'],user_user_test_df['ratings'])))
print("Pearson User User Bias Top10_RMSE_Score : ",math.sqrt(mean_squared_error(user_user_test_df['biased_pearson_Top10'],user_user_test_df['ratings'])))
print("Pearson User User Bias Top2_Precision@10 : ",getAvgPrecisionAt10(user_user_test_df,'biased_pearson_Top2','user_id','movie_id',threashold));
print("Pearson User User Bias Top5_Precision@10 : ",getAvgPrecisionAt10(user_user_test_df,'biased_pearson_Top5','user_id','movie_id',threashold));
print("Pearson User User Bias Top10_Precision@10 : ",getAvgPrecisionAt10(user_user_test_df,'biased_pearson_Top10','user_id','movie_id',threashold));

print("\n")
print("************ Movie Movie Bias ***********************")
print("Pearson Baised Movie Movie Bias Top2_RMSE_Score  : ",math.sqrt(mean_squared_error(test_df['biased_pearson_Top2'],test_df['ratings'])))
print("Pearson Baised Movie Movie Bias Top5_RMSE_Score  : ",math.sqrt(mean_squared_error(test_df['biased_pearson_Top5'],test_df['ratings'])))
print("Pearson Baised Movie Movie Bias Top10_RMSE_Score : ",math.sqrt(mean_squared_error(test_df['biased_pearson_Top10'],test_df['ratings'])))
print("Pearson Baised Movie Movie Bias Top2_Precision@10 : ",getAvgPrecisionAt10(test_df,'biased_pearson_Top2','movie_id','user_id',threashold));
print("Pearson Baised Movie Movie Bias Top5_Precision@10 : ",getAvgPrecisionAt10(test_df,'biased_pearson_Top5','movie_id','user_id',threashold));
print("Pearson Baised Movie Movie Bias Top10_Precision@10 : ",getAvgPrecisionAt10(test_df,'biased_pearson_Top10','movie_id','user_id',threashold));


********************** QUESTION 2 ********************************************************
********************** Threashold  4 ********* for Precision@10 **************
************ user user Avg ***********************
Avg User User Top2_RMSE_Score  :  1.631942860519326
Avg User User Top5_RMSE_Score  :  1.5277683070413524
Avg User User Top10_RMSE_Score :  1.464950852417923
Avg User User Top2_Precision@10 :  0.13232733426967275
Avg User User Top5_Precision@10 :  0.13232733426967275
Avg User User Top10_Precision@10 :  0.13232733426967275


************ Movie Movie Avg ***********************
Avg Movie Movie Top2_RMSE_Score  :  1.8209612845966825
Avg Movie Movie Top5_RMSE_Score  :  1.7784723781942748
Avg Movie Movie User Top10_RMSE_Score :  1.7175243811952132
Avg Movie Movie Top2_Precision@10 :  0.20998117332502944
Avg Movie Movie Top5_Precision@10 :  0.20998117332502944
Avg Movie Movie Top10_Precision@10 :  0.20998117332502944


*********************************************************